In [2]:
from google import genai
import os
from dotenv import load_dotenv
import duckdb
load_dotenv()
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

response = client.models.generate_content(
    model="gemini-2.0-flash", contents="Ge mig en historia om en kanin som bor i skogen på 3 meningar"
)
print(response.text)



Humle var en liten kanin som bodde i en mysig håla under en stor ek i utkanten av skogen. Dagarna spenderade han med att knapra på klöver, hoppa genom gräset och leka kurragömma med sina syskon. När skymningen föll återvände Humle till sin familj i hålan, fylld med berättelser om sina äventyr och doften av vildblommor.



In [ ]:
with duckdb.connect("../ads_data.duckdb") as con: 
    df_mart = con.execute("SELECT * FROM mart.mart_ads").df()
df_test = df_mart[df_mart["occupation_field"] == "Installation, drift, underhåll"]
# df_test.to_csv("sample_ads.csv") # måste göra mer finjusteringar! kan inte läsa allas description direkt!


In [29]:
# descriptions = df_test["description"]

top_employer = df_test.groupby("employer_name")[["vacancies", "description"]].sum().sort_values(by="vacancies", ascending=False).reset_index()
top_employer

,employer_name,vacancies,description
0,eLinked AB,320,Om företaget:\neLinked AB är expansivt konsult...
1,Fordonsakademin Sverige AB,277,Nu söker vi en Reservdelsrådgivare till vår ku...
2,Studentconsulting Sweden AB (Publ),224,Har du ett stort intresse för mekanik och tekn...
3,Aura Personal AB,194,?? Är du redo för nästa steg som Diagnosteknik...
4,Randstad AB,145,Arbetsbeskrivning\nNu söker vi på Randstad en ...
...,...,...,...
835,XR Bolagen AB,1,XR Berg och Maskin i Lidköping söker Reparatör...
836,Work System Sweden AB,1,"Har du ett brinnande intresse för bilar, jobba..."
837,AB Lindex,1,Har du ett intresse för teknik och tycker om a...
838,AB Karlskronahem,1,"Karlskrona som kommun växer, och det gör vi på..."


In [32]:
nr_1 = top_employer[top_employer["employer_name"] == 'eLinked AB']
nr_1["description"].to_csv("top_employer.csv")

In [10]:
df_test.reset_index()

,index,description
0,0,Tedelektro håller på att expandera och vi behö...
1,1,Lagertekniker (Deltid/Extra) – Crona Software\...
2,2,Vi söker en bilmekaniker som vill jobba anting...
3,3,Är du vår nya verkstadstekniker? Vi expander o...
4,4,Plats: Falköping\nStartdatum: 2025-05-15 \nAns...
...,...,...
3769,5049,ONE Nordic bedriver en samhällsviktig verksamh...
3770,5050,Är du teknikintresserad och vill vara en del a...
3771,5097,Om tjänstenVi söker nu mekaniker till våra kun...
3772,5101,Välj inte Elis om one-man-show är din typ av f...


In [34]:
with open("top_employer.csv", "r", encoding='utf-8') as file: 
    ad_text = file.read() 
promt = f"""Du är en rekryterare inom data Installation, drift, underhåll.
Plocka ut 10 av de vanligaste erfarenheterna och kraven som efterfrågas i dessa jobbannonser:
{ad_text}
Output ska vara i detta formatet: 

{{
    "krav": [erfarenhet1, erfarenhet2, ...]
    "meriterande": [meriterande1,meriterande2, ... ]
}}

"""

In [35]:
response = client.models.generate_content(
    model="gemini-2.0-flash", contents=promt
)
print(response.text)

```json
{
    "krav": [
        "Fullständig elutbildning och svenskt el-certifikat (ECY eller motsvarande)",
        "Minst 3-5 års erfarenhet som elektriker/tekniker i Sverige",
        "Körkort B",
        "Flytande svenska i tal och skrift",
        "God datorvana",
        "Erfarenhet av installation, service och underhåll",
        "Felsökning och problemlösning",
        "Arbeta självständigt utefter ritning",
        "Ansvarstagande och klarar hantera flera bollar i luften",
        "God social kompetens och moderna värderingar"
    ],
    "meriterande": [
        "Allmän behörighet (AL)",
        "Erfarenhet som ledande montör eller tidigare egenföretagare inom elinstallation",
        "Tidigare erfarenhet som projektledare, projektsamordnare eller egenföretagare",
        "Utbildningar som Elbam, Bas P/U, heta arbeten, liftkort, fallskydd, AMS, ESA mm.",
        "Erfarenhet inom nyproduktion av villor",
        "Specifik systemkunskap såsom Unison, ASSA, Arks mm.",
        "L

In [ ]:
# import pandas as pd
# data = response.text
# df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in data.items()]))
# df

AttributeError: 'str' object has no attribute 'items'